In [2]:
import pandas
import requests
from bs4 import BeautifulSoup

In [3]:
#Loading the webpage
req  = requests.get("https://pythonhow.com/real-estate/rock-springs-wy/LCWYROCKSPRINGS/")
pageContent = req.content
#Printing pageContent
pageContent

'<!DOCTYPE html>\n\n<html lang="en" style="margin: 0px;overflow:hidden"><script async="" src="./LCWYROCKSPRINGS1_files/beacon.js"></script><script src="chrome-extension://pkljnnogdmlajgaoodihioopfdkpgjgg/Kernel.js?0.3685073930846756"></script><head><meta http-equiv="Content-Type" content="text/html; charset=UTF-8">\n<script type="text/javascript" src="./LCWYROCKSPRINGS1_files/analytics.js"></script>\n<script type="text/javascript">archive_analytics.values.server_name="wwwb-app17.us.archive.org";archive_analytics.values.server_ms=227;</script>\n<link type="text/css" rel="stylesheet" href="./LCWYROCKSPRINGS1_files/banner-styles.css">\n<title>Rock Springs Real Estate | Find Houses &amp; Homes for Sale in Rock Springs, WY</title>\n<meta name="title" content="Rock Springs Real Estate | Find Houses &amp; Homes for Sale in Rock Springs, WY">\n<meta name="description" content="Search Rock Springs real estate property listings to find homes for sale in Rock Springs, WY. Browse houses for sale i

In [4]:
# making the loaded page readable
soup = BeautifulSoup(pageContent,"html.parser")
print(soup.prettify())

<!DOCTYPE html>
<html lang="en" style="margin: 0px;overflow:hidden">
 <script async="" src="./LCWYROCKSPRINGS1_files/beacon.js">
 </script>
 <script src="chrome-extension://pkljnnogdmlajgaoodihioopfdkpgjgg/Kernel.js?0.3685073930846756">
 </script>
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <script src="./LCWYROCKSPRINGS1_files/analytics.js" type="text/javascript">
  </script>
  <script type="text/javascript">
   archive_analytics.values.server_name="wwwb-app17.us.archive.org";archive_analytics.values.server_ms=227;
  </script>
  <link href="./LCWYROCKSPRINGS1_files/banner-styles.css" rel="stylesheet" type="text/css"/>
  <title>
   Rock Springs Real Estate | Find Houses &amp; Homes for Sale in Rock Springs, WY
  </title>
  <meta content="Rock Springs Real Estate | Find Houses &amp; Homes for Sale in Rock Springs, WY" name="title"/>
  <meta content="Search Rock Springs real estate property listings to find homes for sale in Rock Springs, WY. Browse h

In [5]:
#Collecting data from all div blocks i.e the results from webpage
propertyRowDivBlocks=soup.find_all("div",{"class":"propertyRow"})

#Get number of results from each page
len(propertyRowDivBlocks)

10

In [10]:
# Scraping data from the site and adding them to dictionaries
def getListOfDicts(propertyRow):
    listOfDicts=[]
    for item in propertyRow:
        dictionary= {}
        dictionary["Price"] =item.find("h4",{"class":"propPrice"}).text.replace("\n","").replace(" ","")
        addressCollapseDiv = item.find("div",{"class":"primaryDetails"})
        addressList = addressCollapseDiv.find_all("span",{"class":"propAddressCollapse"})
        dictionary["Address"] = addressList[0].text
        dictionary["Locality"] = addressList[1].text
        try:
            dictionary["Beds"]=item.find("span",{"class":"infoBed"}).find("b").text
        except:
            dictionary["Beds"]=None
        try:    
            dictionary["Full Baths"]=item.find("span",{"class":"infoValueFullBath"}).find("b").text
        except:
            dictionary["Full Baths"]=None
        try:    
            dictionary["Half Baths"]=item.find("span",{"class":"infoValueHalfBath"}).find("b").text
        except:
            dictionary["Half Baths"]=None    
        try:    
            dictionary["Area"]=item.find("span",{"class":"infoSqFt"}).find("b").text
        except:
            dictionary["Area"]=None 
        # Getting data for elements without unique identifiers
        for column_group in item.find_all("div",{"class":"columnGroup"}):
            for feature_group,feature_name in zip(column_group.find_all("span",{"class","featureGroup"}),column_group.find_all("span",{"class","featureName"})): 
                if "Lot Size" in feature_group.text:
                    dictionary["Lot Size"]=feature_name.text
        listOfDicts.append(dictionary)
    return listOfDicts   

In [11]:
getListOfDicts(propertyRowDivBlocks)

[{'Address': u'0 Gateway',
  'Area': None,
  'Beds': None,
  'Full Baths': None,
  'Half Baths': None,
  'Locality': u'Rock Springs, WY 82901',
  'Price': u'$725,000'},
 {'Address': u'1003 Winchester Blvd.',
  'Area': None,
  'Beds': u'4',
  'Full Baths': u'4',
  'Half Baths': None,
  'Locality': u'Rock Springs, WY 82901',
  'Lot Size': u'0.21 Acres',
  'Price': u'$452,900'},
 {'Address': u'600 Talladega',
  'Area': u'3,154',
  'Beds': u'5',
  'Full Baths': u'3',
  'Half Baths': None,
  'Locality': u'Rock Springs, WY 82901',
  'Price': u'$396,900'},
 {'Address': u'3239 Spearhead Way',
  'Area': u'3,076',
  'Beds': u'4',
  'Full Baths': u'3',
  'Half Baths': u'1',
  'Locality': u'Rock Springs, WY 82901',
  'Lot Size': u'Under 1/2 Acre, ',
  'Price': u'$389,900'},
 {'Address': u'522 Emerald Street',
  'Area': u'1,172',
  'Beds': u'3',
  'Full Baths': u'3',
  'Half Baths': None,
  'Locality': u'Rock Springs, WY 82901',
  'Lot Size': u'Under 1/2 Acre, ',
  'Price': u'$254,000'},
 {'Address

In [12]:
# Converting list of dicts to dataframe
listOfDicts=getListOfDicts(propertyRowDivBlocks)
scrapedData=pandas.DataFrame(listOfDicts)

In [13]:
scrapedData

Address   Area  Beds Full Baths Half Baths  \
0              0 Gateway   None  None       None       None   
1  1003 Winchester Blvd.   None     4          4       None   
2          600 Talladega  3,154     5          3       None   
3     3239 Spearhead Way  3,076     4          3          1   
4     522 Emerald Street  1,172     3          3       None   
5   1302 Veteran's Drive  1,932     4          2       None   
6       1021 Cypress Cir  1,676     4          3       None   
7         913 Madison Dr  1,344     3          2       None   
8      1344 Teton Street  1,920     3          2       None   
9         4 Minnies Lane  1,664     3          2       None   

                 Locality          Lot Size     Price  
0  Rock Springs, WY 82901               NaN  $725,000  
1  Rock Springs, WY 82901        0.21 Acres  $452,900  
2  Rock Springs, WY 82901               NaN  $396,900  
3  Rock Springs, WY 82901  Under 1/2 Acre,   $389,900  
4  Rock Springs, WY 82901  Under 1/2 Acre,   $254,000  
5  Rock Springs, WY 82901        0.27 Acres  $252,900  
6  Rock Springs, WY 82901  Under 1/2 Acre,   $210,000  
7  Rock Springs, WY 82901  Under 1/2 Acre,   $209,000  
8  Rock Springs, WY 82901  Under 1/2 Acre,   $199,900  
9  Rock Springs, WY 82901        2.02 Acres  $196,900

In [15]:
# Web crawling for all the pages
base_url="https://pythonhow.com/real-estate/rock-springs-wy/LCWYROCKSPRINGS/t=0&s="
finalList=[]
for page in range(0,30,10):
    req=requests.get(base_url+str(page)+".html")
    pageContent=req.content
    soup = BeautifulSoup(pageContent,"html.parser")
    divBlocks=soup.find_all("div",{"class":"propertyRow"})
    dictList=getListOfDicts(divBlocks)
    finalList=finalList+dictList
    

In [16]:
crawlData=pandas.DataFrame(finalList)

In [17]:
crawlData

Address   Area  Beds Full Baths Half Baths  \
0                       0 Gateway   None  None       None       None   
1           1003 Winchester Blvd.   None     4          4       None   
2                   600 Talladega  3,154     5          3       None   
3              3239 Spearhead Way  3,076     4          3          1   
4              522 Emerald Street  1,172     3          3       None   
5            1302 Veteran's Drive  1,932     4          2       None   
6                1021 Cypress Cir  1,676     4          3       None   
7                  913 Madison Dr  1,344     3          2       None   
8               1344 Teton Street  1,920     3          2       None   
9                  4 Minnies Lane  1,664     3          2       None   
10                  9339 Sd 26900  2,560  None       None       None   
11               RR674P13 Hwy 377  2,000  None       None       None   
12                       0 Hwy 41   None  None       None       None   
13                  9339 Sd 26900  2,560  None       None       None   
14                  CR450 Hwy 377   None  None       None       None   
15                  Cr 240 Cr 240  1,398  None       None       None   
16                  RR674 Hwy 377  1,738  None       None       None   
17                 9770a Sd 26900  1,080  None       None       None   
18                  Lot17 CR 2630   None  None       None       None   
19                 Tr12,16 CR 520   None  None       None       None   
20   32575 S Shadow Mountain Road  2,318     3          2       None   
21        32750 S Shangrila Drive  2,120     3          2       None   
22      0000 Black Canyon Highway   None  None       None       None   
23           34775 S CHOLLA Drive  1,220     3          2       None   
24      33403 S. HA-WA-SI TERRACE  2,000     4          2       None   
25          34263 S Bertha Street  2,260     5          2       None   
26            33160 S Canyon Road  1,248     3          2       None   
27        19421 E Todd Evans Road  1,404     3          2       None   
28             18688 E AGUA Vista   None  None       None       None   
29  50600 N Old Black Canyon Road   None  None       None       None   
30      20101 E SQUAW VALLEY Road   None  None       None       None   
31            33259 S Canyon Road  1,056     3          1       None   
32          34558 S ROADRUNNER RD    784     2          1       None   
33       19260 E Scenic Loop Road   None  None       None       None   
34           19000 E MAREN Avenue   None  None       None       None   
35          19350 E SAGUARO Drive   None  None       None       None   
36         20650 E Amethyst Place   None  None       None       None   

                       Locality          Lot Size       Price  
0        Rock Springs, WY 82901               NaN    $725,000  
1        Rock Springs, WY 82901        0.21 Acres    $452,900  
2        Rock Springs, WY 82901               NaN    $396,900  
3        Rock Springs, WY 82901  Under 1/2 Acre,     $389,900  
4        Rock Springs, WY 82901  Under 1/2 Acre,     $254,000  
5        Rock Springs, WY 82901        0.27 Acres    $252,900  
6        Rock Springs, WY 82901  Under 1/2 Acre,     $210,000  
7        Rock Springs, WY 82901  Under 1/2 Acre,     $209,000  
8        Rock Springs, WY 82901  Under 1/2 Acre,     $199,900  
9        Rock Springs, WY 82901        2.02 Acres    $196,900  
10        Rocksprings, TX 78880               NaN  $1,700,000  
11        Rocksprings, TX 78880               NaN  $1,100,000  
12        Rocksprings, TX 78880               NaN  $1,080,000  
13        Rocksprings, TX 78880               NaN    $908,350  
14        Rocksprings, TX 78880               NaN    $905,000  
15        Rocksprings, TX 78880               NaN    $695,000  
16        Rocksprings, TX 78880               NaN    $605,000  
17        Rocksprings, TX 78880               NaN    $559,805  
18        Rocksprings, TX 78880               NaN    $504,000  
19       

In [18]:
crawlData.to_csv("CrawlData.csv")